In [51]:
from fastlane_bot import Bot, Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
from fastlane_bot.testing import *
import itertools as it
import collections as cl
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.8.1 (05/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
Version = 3-b2.1 [requirements >= 3.0 is met]


# Mainnet Server [NB030]

In [36]:
bot     = Bot()
CCm     = bot.get_curves()

In [36]:
pairs0  = CCm.pairs(standardize=False)
pairs   = CCm.pairs(standardize=True)
pairsc  = {c.pairo.primary for c in CCm if c.P("exchange")=="carbon_v1"}

## Overall market [NOTEST]

In [37]:
print(f"Total pairs:    {len(pairs0):4}")
print(f"Primary pairs:  {len(pairs):4}")
print(f"...carbon:      {len(pairsc):4}")
print(f"Tokens:         {len(CCm.tokens()):4}")
print(f"Curves:         {len(CCm):4}")

Total pairs:     666
Primary pairs:   637
...carbon:        25
Tokens:          512
Curves:          810


## By pair [NOTEST]

### All pairs

In [43]:
cbp0 = {pair: [c for c in CCm.bypairs(pair)] for pair in CCm.pairs()} # curves by (primary) pair
nbp0 = {pair: len(cc) for pair,cc in cbp0.items()}
assert len(cbp0) == len(CCm.pairs())
assert set(cbp0) == CCm.pairs()

### Only those with >1 curves

In [42]:
cbp = {pair: cc for pair, cc in cbp0.items() if len(cc)>1}
nbp = {pair: len(cc) for pair,cc in cbp.items()}
print(f"Pairs with >1 curves:  {len(cbp)}")
print(f"Curves in those:       {sum(nbp.values())}")
print(f"Average curves/pair:   {sum(nbp.values())/len(cbp):.1f}")

Pairs with >1 curves:  110
Curves in those:       283
Average curves/pair:   2.6


### x=0 or y=0

In [41]:
xis0 = {c.cid: (c.x, c.y) for c in CCm if c.x==0}
yis0 = {c.cid: (c.x, c.y) for c in CCm if c.y==0}
assert len(xis0) == 0 # set loglevel debug to see removal of curves
assert len(yis0) == 0

### Prices

#### All

In [30]:
prices_da = {pair: 
             [(Pair.n(pair), c.primaryp(), c.cid, c.cid[-8:], c.P("exchange"), c.buysell(verbose=False), 
               c.buysell(verbose=True, withprice=True)) 
                    for c in cc] for pair, cc in cbp.items()}
#prices_f

#### Only for pairs that have at least on Carbon pair

In [46]:
prices_d = {pair: l for pair,l in prices_da.items() if pair in pairsc}
prices_l = list(it.chain(*prices_d.values()))

In [57]:
curves_by_pair = list(cl.Counter([r[0] for r in prices_l]).items())
curves_by_pair = sorted(curves_by_pair, key=lambda x: x[1], reverse=True)
curves_by_pair

[('WETH/USDC', 18),
 ('BNT/WETH', 12),
 ('BNT/vBNT', 10),
 ('USDT/USDC', 8),
 ('WBTC/WETH', 5),
 ('LINK/USDT', 5),
 ('WBTC/USDT', 4),
 ('BNT/USDC', 4),
 ('WETH/DAI', 3),
 ('LINK/USDC', 3),
 ('DAI/USDC', 3),
 ('WETH/USDT', 3),
 ('DAI/USDT', 3),
 ('PEPE/WETH', 3),
 ('LYXe/USDC', 2),
 ('rETH/WETH', 2),
 ('0x0/WETH', 2),
 ('WBTC/USDC', 2),
 ('ARB/MATIC', 2),
 ('TSUKA/USDC', 2),
 ('stETH/WETH', 2)]

In [62]:
# for pair, _ in curves_by_pair:
#     print(f"# #### {pair}\n\npricedf.loc['{pair}']\n\n")

#### Dataframe

In [63]:
pricedf0 = pd.DataFrame(prices_l, columns="pair,price,cid,cid0,exchange,bs, bsv".split(","))
pricedf = pricedf0.drop('cid', axis=1)
pricedf = pricedf.set_index(["pair", "exchange", "cid0"])
pricedf

price  bs  \
pair      exchange     cid0                         
WBTC/WETH uniswap_v3   466       1.501514e+01  bs   
          sushiswap_v2 804       1.490428e+01  bs   
          uniswap_v3   478       1.531413e+01  bs   
          carbon_v1    709362-0  1.539975e+01   s   
                       709362-1  1.428571e+01   b   
...                                       ...  ..   
BNT/USDC  carbon_v1    480199-0  2.000000e+00   s   
          bancor_v3    720       4.837881e-01  bs   
PEPE/WETH carbon_v1    440620-0  3.500000e-07   b   
                       440620-1  4.000000e-07   s   
                       440621-1  4.500000e-07   s   

                                                                 bsv  
pair      exchange     cid0                                           
WBTC/WETH uniswap_v3   466       buy-sell-WBTC @ 15.02 WETH per WBTC  
          sushiswap_v2 804       buy-sell-WBTC @ 14.90 WETH per WBTC  
          uniswap_v3   478       buy-sell-WBTC @ 15.31 WETH per WBTC  
          carbon_v1    709362-0      sell-WBTC @ 15.40 WETH per WBTC  
                       709362-1       buy-WBTC @ 14.29 WETH per WBTC  
...                                                              ...  
BNT/USDC  carbon_v1    480199-0         sell-BNT @ 2.00 USDC per BNT  
          bancor_v3    720          buy-sell-BNT @ 0.48 USDC per BNT  
PEPE/WETH carbon_v1    440620-0        buy-PEPE @ 0.00 WETH per PEPE  
                       440620-1       sell-PEPE @ 0.00 WETH per PEPE  
                       440621-1       sell-PEPE @ 0.00 WETH per PEPE  

[98 rows x 3 columns]

### Individual frames

#### WETH/USDC

In [64]:
pricedf.loc['WETH/USDC']

price  bs                                    bsv
exchange     cid0                                                            
uniswap_v2   255       1984.494958  bs  buy-sell-WETH @ 1984.49 USDC per WETH
uniswap_v3   593       1976.083421  bs  buy-sell-WETH @ 1976.08 USDC per WETH
sushiswap_v2 803       1982.349633  bs  buy-sell-WETH @ 1982.35 USDC per WETH
uniswap_v3   346       1988.906177  bs  buy-sell-WETH @ 1988.91 USDC per WETH
carbon_v1    057285-1  2099.999790   s      sell-WETH @ 2100.00 USDC per WETH
             057292-0  1853.408818   b       buy-WETH @ 1853.41 USDC per WETH
             057292-1  2000.000000   s      sell-WETH @ 2000.00 USDC per WETH
             057296-0  1930.000193   b       buy-WETH @ 1930.00 USDC per WETH
             057296-1  1950.000195   s      sell-WETH @ 1950.00 USDC per WETH
             057299-1  1940.000000   s      sell-WETH @ 1940.00 USDC per WETH
             057306-0  1405.000140   b       buy-WETH @ 1405.00 USDC per WETH
             057315-1  2300.000000   s      sell-WETH @ 2300.00 USDC per WETH
             057331-0  1800.000000   b       buy-WETH @ 1800.00 USDC per WETH
             057334-0  1700.000170   b       buy-WETH @ 1700.00 USDC per WETH
             057334-1  1999.999800   s      sell-WETH @ 2000.00 USDC per WETH
             057337-0  1850.000000   b       buy-WETH @ 1850.00 USDC per WETH
             057339-0  1800.000000   b       buy-WETH @ 1800.00 USDC per WETH
             057343-1  1989.999801   s      sell-WETH @ 1990.00 USDC per WETH

#### BNT/WETH

In [65]:
pricedf.loc['BNT/WETH']

price  bs                               bsv
exchange   cid0                                                    
carbon_v1  326030-0  0.000253   s      sell-BNT @ 0.00 WETH per BNT
           326030-1  0.000200   b       buy-BNT @ 0.00 WETH per BNT
uniswap_v2 290       0.000248  bs  buy-sell-BNT @ 0.00 WETH per BNT
carbon_v1  326034-0  0.000200   b       buy-BNT @ 0.00 WETH per BNT
           326034-1  0.002100   s      sell-BNT @ 0.00 WETH per BNT
           326031-0  0.000200   b       buy-BNT @ 0.00 WETH per BNT
           326031-1  0.002000   s      sell-BNT @ 0.00 WETH per BNT
bancor_v2  623       0.000245  bs  buy-sell-BNT @ 0.00 WETH per BNT
carbon_v1  326076-0  0.000253   b       buy-BNT @ 0.00 WETH per BNT
           326077-0  0.000251   b       buy-BNT @ 0.00 WETH per BNT
           326077-1  0.000258   s      sell-BNT @ 0.00 WETH per BNT
bancor_v3  704       0.000248  bs  buy-sell-BNT @ 0.00 WETH per BNT

#### BNT/vBNT

In [66]:
pricedf.loc['BNT/vBNT']

price  bs                               bsv
exchange  cid0                                                    
carbon_v1 748990-1  0.952381   b       buy-BNT @ 0.95 vBNT per BNT
bancor_v3 742       1.252518  bs  buy-sell-BNT @ 1.25 vBNT per BNT
bancor_v2 653       1.329008  bs  buy-sell-BNT @ 1.33 vBNT per BNT
carbon_v1 748965-1  0.909091   b       buy-BNT @ 0.91 vBNT per BNT
          748976-0  1.331480   s      sell-BNT @ 1.33 vBNT per BNT
          748976-1  1.111111   b       buy-BNT @ 1.11 vBNT per BNT
          748950-0  0.940000   b       buy-BNT @ 0.94 vBNT per BNT
          748977-0  1.428571   s      sell-BNT @ 1.43 vBNT per BNT
          748977-1  1.250000   b       buy-BNT @ 1.25 vBNT per BNT
          748966-1  1.000000   b       buy-BNT @ 1.00 vBNT per BNT

#### USDT/USDC

In [67]:
pricedf.loc['USDT/USDC']

price  bs                                 bsv
exchange     cid0                                                      
uniswap_v3   357       1.002654  bs  buy-sell-USDT @ 1.00 USDC per USDT
uniswap_v2   246       1.000174  bs  buy-sell-USDT @ 1.00 USDC per USDT
uniswap_v3   486       1.000580  bs  buy-sell-USDT @ 1.00 USDC per USDT
sushiswap_v2 805       0.998806  bs  buy-sell-USDT @ 1.00 USDC per USDT
carbon_v1    634391-0  1.001001   s      sell-USDT @ 1.00 USDC per USDT
             634391-1  1.000690   b       buy-USDT @ 1.00 USDC per USDT
             634371-0  1.005025   s      sell-USDT @ 1.01 USDC per USDT
             634371-1  0.995025   b       buy-USDT @ 1.00 USDC per USDT

#### WBTC/WETH

In [68]:
pricedf.loc['WBTC/WETH']

price  bs                                  bsv
exchange     cid0                                                        
uniswap_v3   466       15.015145  bs  buy-sell-WBTC @ 15.02 WETH per WBTC
sushiswap_v2 804       14.904278  bs  buy-sell-WBTC @ 14.90 WETH per WBTC
uniswap_v3   478       15.314129  bs  buy-sell-WBTC @ 15.31 WETH per WBTC
carbon_v1    709362-0  15.399750   s      sell-WBTC @ 15.40 WETH per WBTC
             709362-1  14.285714   b       buy-WBTC @ 14.29 WETH per WBTC

#### LINK/USDT

In [69]:
pricedf.loc['LINK/USDT']

price  bs                                 bsv
exchange     cid0                                                      
uniswap_v2   171       7.328775  bs  buy-sell-LINK @ 7.33 USDT per LINK
uniswap_v3   549       7.169997  bs  buy-sell-LINK @ 7.17 USDT per LINK
sushiswap_v2 791       7.123545  bs  buy-sell-LINK @ 7.12 USDT per LINK
carbon_v1    960408-0  7.100000   b       buy-LINK @ 7.10 USDT per LINK
             960408-1  7.700000   s      sell-LINK @ 7.70 USDT per LINK

#### WBTC/USDT

In [70]:
pricedf.loc['WBTC/USDT']

price  bs  \
exchange     cid0                         
uniswap_v2   183       29095.896331  bs   
sushiswap_v2 814       29004.432929  bs   
carbon_v1    920820-0  28980.881784   b   
             920820-1  29500.000000   s   

                                                          bsv  
exchange     cid0                                              
uniswap_v2   183       buy-sell-WBTC @ 29095.90 USDT per WBTC  
sushiswap_v2 814       buy-sell-WBTC @ 29004.43 USDT per WBTC  
carbon_v1    920820-0       buy-WBTC @ 28980.88 USDT per WBTC  
             920820-1      sell-WBTC @ 29500.00 USDT per WBTC

#### BNT/USDC

In [71]:
pricedf.loc['BNT/USDC']

,,price,bs,bsv
exchange,cid0,,,
carbon_v1,480202-1,1.480041,s,sell-BNT @ 1.48 USDC per BNT
bancor_v2,652,0.473736,bs,buy-sell-BNT @ 0.47 USDC per BNT
carbon_v1,480199-0,2.000000,s,sell-BNT @ 2.00 USDC per BNT
bancor_v3,720,0.483788,bs,buy-sell-BNT @ 0.48 USDC per BNT


#### WETH/DAI

In [72]:
pricedf.loc['WETH/DAI']

,,price,bs,bsv
exchange,cid0,,,
carbon_v1,211457-1,1944.999806,s,sell-WETH @ 1945.00 DAI per WETH
uniswap_v3,594,1979.616549,bs,buy-sell-WETH @ 1979.62 DAI per WETH
sushiswap_v2,817,1981.145633,bs,buy-sell-WETH @ 1981.15 DAI per WETH


#### LINK/USDC

In [73]:
pricedf.loc['LINK/USDC']

,,price,bs,bsv
exchange,cid0,,,
uniswap_v2,176,6.794102,bs,buy-sell-LINK @ 6.79 USDC per LINK
sushiswap_v2,792,6.935941,bs,buy-sell-LINK @ 6.94 USDC per LINK
carbon_v1,497903-1,7.750000,s,sell-LINK @ 7.75 USDC per LINK


#### DAI/USDC

In [74]:
pricedf.loc['DAI/USDC']

price  bs                               bsv
exchange     cid0                                                    
carbon_v1    845828-0  1.001001   s      sell-DAI @ 1.00 USDC per DAI
             845828-1  0.999001   b       buy-DAI @ 1.00 USDC per DAI
sushiswap_v2 839       1.003169  bs  buy-sell-DAI @ 1.00 USDC per DAI

#### WETH/USDT

In [75]:
pricedf.loc['WETH/USDT']

,,price,bs,bsv
exchange,cid0,,,
uniswap_v2,256,1989.408342,bs,buy-sell-WETH @ 1989.41 USDT per WETH
sushiswap_v2,840,1985.382699,bs,buy-sell-WETH @ 1985.38 USDT per WETH
carbon_v1,691656-0,1900.000190,b,buy-WETH @ 1900.00 USDT per WETH


#### DAI/USDT

In [76]:
pricedf.loc['DAI/USDT']

price  bs                               bsv
exchange     cid0                                                    
carbon_v1    268742-0  0.995000   b       buy-DAI @ 1.00 USDT per DAI
             268742-1  1.005000   s      sell-DAI @ 1.00 USDT per DAI
sushiswap_v2 795       0.994182  bs  buy-sell-DAI @ 0.99 USDT per DAI

#### PEPE/WETH

In [77]:
pricedf.loc['PEPE/WETH']

price bs                             bsv
exchange  cid0                                                     
carbon_v1 440620-0  3.500000e-07  b   buy-PEPE @ 0.00 WETH per PEPE
          440620-1  4.000000e-07  s  sell-PEPE @ 0.00 WETH per PEPE
          440621-1  4.500000e-07  s  sell-PEPE @ 0.00 WETH per PEPE

#### LYXe/USDC

In [78]:
pricedf.loc['LYXe/USDC']

,,price,bs,bsv
exchange,cid0,,,
uniswap_v3,558,14.710999,bs,buy-sell-LYXe @ 14.71 USDC per LYXe
carbon_v1,652071-1,15.999998,s,sell-LYXe @ 16.00 USDC per LYXe


#### rETH/WETH

In [79]:
pricedf.loc['rETH/WETH']

,,price,bs,bsv
exchange,cid0,,,
sushiswap_v2,833,1.237861,bs,buy-sell-rETH @ 1.24 WETH per rETH
carbon_v1,903115-0,1.069000,b,buy-rETH @ 1.07 WETH per rETH


#### 0x0/WETH

In [80]:
pricedf.loc['0x0/WETH']

price bs                           bsv
exchange  cid0                                               
carbon_v1 132277-0  0.000013  b   buy-0x0 @ 0.00 WETH per 0x0
          132277-1  0.000014  s  sell-0x0 @ 0.00 WETH per 0x0

#### WBTC/USDC

In [81]:
pricedf.loc['WBTC/USDC']

,,price,bs,bsv
exchange,cid0,,,
uniswap_v2,332,29690.51548,bs,buy-sell-WBTC @ 29690.52 USDC per WBTC
carbon_v1,537493-0,28000.00000,b,buy-WBTC @ 28000.00 USDC per WBTC


#### ARB/MATIC

In [82]:
pricedf.loc['ARB/MATIC']

price bs                            bsv
exchange  cid0                                                
carbon_v1 806240-0  1.507045  s  sell-ARB @ 1.51 MATIC per ARB
          806240-1  1.428571  b   buy-ARB @ 1.43 MATIC per ARB

#### TSUKA/USDC

In [83]:
pricedf.loc['TSUKA/USDC']

,,price,bs,bsv
exchange,cid0,,,
uniswap_v2,254,0.030762,bs,buy-sell-TSUKA @ 0.03 USDC per TSUKA
carbon_v1,017697-1,0.120000,s,sell-TSUKA @ 0.12 USDC per TSUKA


#### stETH/WETH

In [84]:
pricedf.loc['stETH/WETH']

price bs                               bsv
exchange  cid0                                                   
carbon_v1 422914-0  1.010101  s  sell-stETH @ 1.01 WETH per stETH
          422914-1  0.990099  b   buy-stETH @ 0.99 WETH per stETH

In [85]:
raise

RuntimeError: No active exception to reraise

## Execution [NOTEST]

### Configuration

- `flt`: flashloanable tokens
- `loglevel`: `LL_DEBUG` , `LL_INFO` `LL_WARN` `LL_ERR`

In [13]:
flt = [T.USDC]
C = Config.new(config=Config.CONFIG_TENDERLY, loglevel=Config.LL_INFO)

In [14]:
bot = CarbonBot(ConfigObj=C)

NameError: name 'CarbonBot' is not defined

### Database update [Tenderly specific]

In [ ]:
# provided here for convenience; must be commented out for tests
bot.update(drop_tables=True, top_n=10, only_carbon=False)

### Execution

In [ ]:
bot.run(flashloan_tokens=flt, mode=bot.RUN_SINGLE)

## Execution analysis [NOTEST]

In [ ]:
CCm = bot.get_curves()

### Arbitrage opportunities

In [ ]:
ops = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ARBOPPS)
ops

### Route struct

In [ ]:
try:
    route_struct = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ROUTE)
except bot.NoArbAvailable as e:
    print(f"[NoArbAvailable] {e}")
    route_struct = None
route_struct

### Orderering info

In [ ]:
try:
    ordinfo = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ORDINFO)
    flashloan_amount = ordinfo[1]
    flashloan_token_address = ordinfo[2]
    print(f"Flashloan: {flashloan_amount} [{flashloan_token_address}]")
except bot.NoArbAvailable as e:
    print(f"[NoArbAvailable] {e}")
    ordinfo = None
ordinfo

## Market analysis [NOTEST]

### Overall market

In [ ]:
exch0 = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
#print("Exchanges:", exch0)
print("---")
for xc in exch0:
    print(f"{xc+':':16} {len(CCm.byparams(exchange=xc)):4}")

### Pair

In [ ]:
pair = f"{T.ECO}/{T.USDC}"

In [ ]:
CCp = CCm.bypairs(pair)
exch = {c.P("exchange") for c in CCp}
print("pair:           ", pair)
print("curves:         ", len(CCp))
print("exchanges:      ", exch)
for xc in exch:
    c = CCp.byparams(exchange=xc)[0]
    print(f"{xc+':':16} {c.p:.4f} {1/c.p:.4f}")

## Technical [NOTEST]

### Validation and assertions

In [ ]:
assert C.DATABASE == C.DATABASE_POSTGRES
assert C.POSTGRES_DB == "tenderly"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"
assert C.w3.provider.endpoint_uri.startswith("https://rpc.tenderly.co/fork/")
assert bot.db.carbon_controller.address == '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1'

### Tenderly shell commands

Run those commands in a shell if there are Tenderly connection issues

In [ ]:
C_nw = ConfigNetwork.new(network=ConfigNetwork.NETWORK_TENDERLY)
c1, c2 = C_nw.shellcommand().splitlines()
print(c1)
print(c2)
# !{c1}
# !{c2}